In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
from datetime import datetime
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.titlesize'] = 18

In [13]:
t = pd.read_csv('downloaded-logs-20230406-192033.csv', delimiter=',')
t.head()

,insertId,jsonPayload.blockHash,jsonPayload.contentLength,jsonPayload.headSlot,jsonPayload.idArg,jsonPayload.level,jsonPayload.method,jsonPayload.mevBoostV,jsonPayload.msIntoSlot,jsonPayload.msg,...,receivedLocation,resource.labels.cluster_name,resource.labels.container_name,resource.labels.location,resource.labels.namespace_name,resource.labels.pod_name,resource.labels.project_id,resource.type,severity,timestamp
0,qvhwijyfzotnd0h0,0xe4976f1a133a1d86d8742c9f38e2f35556c12dd7ccb2...,44389,6166311,NaN,error,getPayload,v1.5.0,7538,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-n4kg7,ultra-sound-relay,k8s_container,ERROR,2023-04-06T22:22:42Z
1,nwxnobpohvz42d5m,0xe4976f1a133a1d86d8742c9f38e2f35556c12dd7ccb2...,44389,6166311,NaN,error,getPayload,v1.5.0,7261,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-mszc9,ultra-sound-relay,k8s_container,ERROR,2023-04-06T22:22:42Z
2,kw8byrtskezcqjda,0xe4976f1a133a1d86d8742c9f38e2f35556c12dd7ccb2...,44389,6166311,NaN,error,getPayload,v1.5.0,7118,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-vm84v,ultra-sound-relay,k8s_container,ERROR,2023-04-06T22:22:42Z
3,ft16r8bnucpyo1rd,0xe4976f1a133a1d86d8742c9f38e2f35556c12dd7ccb2...,44389,6166311,NaN,error,getPayload,v1.5.0,7401,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-vm84v,ultra-sound-relay,k8s_container,ERROR,2023-04-06T22:22:42Z
4,5h3yj9m7sso08b3c,0xe4976f1a133a1d86d8742c9f38e2f35556c12dd7ccb2...,44389,6166311,NaN,error,getPayload,v1.5.0,7676,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-n4kg7,ultra-sound-relay,k8s_container,ERROR,2023-04-06T22:22:42Z


In [3]:
t = t[['jsonPayload.slot', 'jsonPayload.msIntoSlot']]
t.head()

,jsonPayload.slot,jsonPayload.msIntoSlot
0,6166311,7538
1,6166311,7261
2,6166311,7118
3,6166311,7401
4,6166311,7676


In [4]:
len(t)

320

In [5]:
t = t.sort_values('jsonPayload.msIntoSlot', ascending=True).drop_duplicates('jsonPayload.slot').sort_index()
len(t)

68

In [6]:
n = pd.read_csv('downloaded-logs-20230406-192256.csv', delimiter=',')
nslots = list(set(n['jsonPayload.slot']))
nslots

[6159970,
 6161444,
 6159717,
 6164230,
 6162185,
 6164651,
 6164076,
 6166509,
 6159856,
 6159122,
 6163892,
 6158902,
 6161114,
 6164317,
 6161054,
 6159295]

In [7]:
t = t[~t['jsonPayload.slot'].isin(nslots)]
len(t)

66

In [8]:
t

,jsonPayload.slot,jsonPayload.msIntoSlot
2,6166311,7118
6,6166134,4093
9,6165969,8139
18,6165651,4971
20,6165621,4033
...,...,...
296,6159358,6166
304,6159339,8841
309,6159188,7725
314,6158759,4831


In [9]:
res = []
for s in t['jsonPayload.slot']:
    time.sleep(0.1)
    url = 'http://57.128.92.65:3500/eth/v1/beacon/headers/{}'.format(s)
    headers = {'content-type': 'application/json'}
    r = requests.get(url, headers=headers)
    try: 
        code = r.json()['code']
        print(s, False)
        res.append((s, False))
    except KeyError:
        print(s, True)
        res.append((s, True))
res = np.asarray(res)

6166311 True
6166134 True
6165969 True
6165651 True
6165621 True
6165608 True
6165462 True
6165280 True
6164747 True
6164675 False
6164576 False
6164414 False
6164393 True
6163925 True
6163842 True
6163750 True
6163617 True
6163614 True
6163595 True
6163590 False
6163538 True
6163448 True
6163414 False
6163399 True
6163338 True
6163229 True
6163226 False
6163208 False
6163125 True
6162885 True
6162855 True
6162624 True
6162476 True
6162409 False
6162290 False
6162268 True
6162225 True
6161875 True
6161720 True
6161703 True
6161656 True
6161563 True
6161437 True
6161253 True
6161081 False
6160918 True
6160769 False
6160765 True
6160753 True
6160634 True
6160592 True
6160417 True
6160120 True
6160103 True
6159945 True
6159894 True
6159688 True
6159669 True
6159618 True
6159401 True
6159384 True
6159358 True
6159339 True
6159188 True
6158759 True
6158720 False


In [10]:
missed = [v[0] for v in res if v[1] == 0]

In [11]:
t[t['jsonPayload.slot'].isin(missed)]

,jsonPayload.slot,jsonPayload.msIntoSlot
50,6164675,4200
54,6164576,7494
55,6164414,5821
95,6163590,5644
105,6163414,9223372036854776000
120,6163226,5818
129,6163208,5213
163,6162409,5145
166,6162290,5851
234,6161081,14500


In [23]:
11/2806

0.003920171062009979

## after 6164576

In [27]:
t = pd.read_csv('downloaded-logs-20230408-090334.csv', delimiter=',')
t.head()

,insertId,jsonPayload.blockHash,jsonPayload.contentLength,jsonPayload.headSlot,jsonPayload.idArg,jsonPayload.level,jsonPayload.method,jsonPayload.mevBoostV,jsonPayload.msIntoSlot,jsonPayload.msg,...,receivedLocation,resource.labels.cluster_name,resource.labels.container_name,resource.labels.location,resource.labels.namespace_name,resource.labels.pod_name,resource.labels.project_id,resource.type,severity,timestamp
0,4yyfbsyhtwht56ut,0xc828a645499a94f5236a1ea4f39de6e1f0967298d71b...,68134,6177861,NaN,error,getPayload,-,4834,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-n4kg7,ultra-sound-relay,k8s_container,ERROR,2023-04-08T12:52:39Z
1,xxl87xchg79dx3mu,0xa0965f589fab0d18ff1565df96177a70353035af6690...,84795,6177849,NaN,error,getPayload,v1.5.0,4133,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-bxkzb,ultra-sound-relay,k8s_container,ERROR,2023-04-08T12:50:15Z
2,w7ydq33t2repciig,0xa0965f589fab0d18ff1565df96177a70353035af6690...,84795,6177849,NaN,error,getPayload,v1.5.0,4642,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-n4kg7,ultra-sound-relay,k8s_container,ERROR,2023-04-08T12:50:15Z
3,dbigxg8si4r2058r,0xa0965f589fab0d18ff1565df96177a70353035af6690...,84795,6177849,NaN,error,getPayload,v1.5.0,4304,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-bxkzb,ultra-sound-relay,k8s_container,ERROR,2023-04-08T12:50:15Z
4,a7klvpagx7d9b6kj,0xa0965f589fab0d18ff1565df96177a70353035af6690...,84795,6177849,NaN,error,getPayload,v1.5.0,4475,timestamp too late,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-75f6cfd5c5-n4kg7,ultra-sound-relay,k8s_container,ERROR,2023-04-08T12:50:15Z


In [28]:
t['jsonPayload.slot'].min()

6164576

In [29]:
t = t[['jsonPayload.slot', 'jsonPayload.msIntoSlot']]
t.head()

,jsonPayload.slot,jsonPayload.msIntoSlot
0,6177861,4834
1,6177849,4133
2,6177849,4642
3,6177849,4304
4,6177849,4475


In [30]:
len(t)

343

In [31]:
t = t.sort_values('jsonPayload.msIntoSlot', ascending=True).drop_duplicates('jsonPayload.slot').sort_index()
len(t)

97

In [32]:
t.head()

,jsonPayload.slot,jsonPayload.msIntoSlot
0,6177861,4834
1,6177849,4133
7,6177768,4651
9,6177420,4685
11,6177403,5099


In [33]:
n = pd.read_csv('downloaded-logs-20230408-090358.csv', delimiter=',')
nslots = list(set(n['jsonPayload.slot']))
nslots

[6170881,
 6171394,
 6172292,
 6176135,
 6174728,
 6167179,
 6174092,
 6173587,
 6170775,
 6171928,
 6176665,
 6176285,
 6170145,
 6170146,
 6176294,
 6164651,
 6175021,
 6170295,
 6170296,
 6174394,
 6170173,
 6173503,
 6177092,
 6173636,
 6176198,
 6177488,
 6167902,
 6177123,
 6166509,
 6166895,
 6170608,
 6173567]

In [34]:
t = t[~t['jsonPayload.slot'].isin(nslots)]
len(t)

93

In [35]:
t

,jsonPayload.slot,jsonPayload.msIntoSlot
0,6177861,4834
1,6177849,4133
7,6177768,4651
9,6177420,4685
11,6177403,5099
...,...,...
320,6165462,6936
324,6165280,6769
326,6164747,4026
338,6164675,4200


In [36]:
res = []
for s in t['jsonPayload.slot']:
    time.sleep(0.1)
    url = 'http://57.128.92.65:3500/eth/v1/beacon/headers/{}'.format(s)
    headers = {'content-type': 'application/json'}
    r = requests.get(url, headers=headers)
    try: 
        code = r.json()['code']
        print(s, False)
        res.append((s, False))
    except KeyError:
        print(s, True)
        res.append((s, True))
res = np.asarray(res)

6177861 True
6177849 True
6177768 True
6177420 True
6177403 False
6177248 False
6177180 True
6177129 True
6177056 True
6176984 True
6176919 True
6176494 True
6176123 False
6176104 True
6176069 True
6175987 False
6175884 True
6175871 True
6175788 True
6175728 True
6175572 True
6175320 True
6175258 False
6175209 True
6175187 True
6175042 True
6174710 True
6174709 True
6174698 True
6174505 True
6174393 True
6174387 True
6174234 True
6173993 True
6173983 True
6173966 True
6173912 True
6173699 False
6173528 True
6173439 True
6173312 True
6173125 True
6173093 True
6173058 True
6173039 True
6172659 True
6172379 True
6171951 True
6171939 True
6171935 True
6171460 True
6171447 True
6171411 True
6171186 True
6170935 True
6170830 True
6170695 True
6170538 True
6170370 True
6170312 True
6170276 True
6170242 False
6170180 False
6170062 False
6170053 False
6169872 True
6169585 False
6169526 True
6169237 True
6169000 True
6168977 True
6168832 True
6168542 True
6167914 True
6167670 True
6167640 True
6

In [37]:
missed = [v[0] for v in res if v[1] == 0]

In [38]:
t[t['jsonPayload.slot'].isin(missed)]

,jsonPayload.slot,jsonPayload.msIntoSlot
11,6177403,5099
18,6177248,4208
41,6176123,4550
56,6175987,5893
70,6175258,4325
124,6173699,6408
214,6170242,4066
218,6170180,5635
221,6170062,5080
234,6170053,5447


In [23]:
11/2806

0.003920171062009979

## after 6178416

In [39]:
t = pd.read_csv('downloaded-logs-20230409-210035.csv', delimiter=',')
t.head()

,insertId,jsonPayload.blockHash,jsonPayload.contentLength,jsonPayload.headSlot,jsonPayload.idArg,jsonPayload.level,jsonPayload.method,jsonPayload.mevBoostV,jsonPayload.msIntoSlot,jsonPayload.msg,...,receiveTimestamp,resource.labels.cluster_name,resource.labels.container_name,resource.labels.location,resource.labels.namespace_name,resource.labels.pod_name,resource.labels.project_id,resource.type,severity,timestamp
0,v1sly7dz1hrdzyji,0x1e3b22df72902f42015133953dad5f8fb2a4ec6e37f1...,84762,6188539,NaN,error,getPayload,v1.5.0,6119,timestamp too late,...,2023-04-10T00:28:18.008157027Z,midori,proposer-api,europe-west1-b,default,proposer-api-7d4b67476f-wq2fp,ultra-sound-relay,k8s_container,ERROR,2023-04-10T00:28:17Z
1,hl1ckxjfh1je8mxx,0x1e3b22df72902f42015133953dad5f8fb2a4ec6e37f1...,84762,6188539,NaN,error,getPayload,v1.5.0,6287,timestamp too late,...,2023-04-10T00:28:22.163535837Z,midori,proposer-api,europe-west1-b,default,proposer-api-7d4b67476f-ql88l,ultra-sound-relay,k8s_container,ERROR,2023-04-10T00:28:17Z
2,0bkmlcvduylmcwrt,0x1e3b22df72902f42015133953dad5f8fb2a4ec6e37f1...,84762,6188539,NaN,error,getPayload,v1.5.0,6459,timestamp too late,...,2023-04-10T00:28:18.006965548Z,midori,proposer-api,europe-west1-b,default,proposer-api-7d4b67476f-vc6bz,ultra-sound-relay,k8s_container,ERROR,2023-04-10T00:28:17Z
3,m3xi5mthedvc22np,0x1e3b22df72902f42015133953dad5f8fb2a4ec6e37f1...,84762,6188539,NaN,error,getPayload,v1.5.0,5943,timestamp too late,...,2023-04-10T00:28:18.008157027Z,midori,proposer-api,europe-west1-b,default,proposer-api-7d4b67476f-wq2fp,ultra-sound-relay,k8s_container,ERROR,2023-04-10T00:28:16Z
4,29gg37q9fdglk391,0x1e3b22df72902f42015133953dad5f8fb2a4ec6e37f1...,84762,6188539,NaN,error,getPayload,v1.5.0,5759,timestamp too late,...,2023-04-10T00:28:17.135289253Z,midori,proposer-api,europe-west1-b,default,proposer-api-7d4b67476f-ql88l,ultra-sound-relay,k8s_container,ERROR,2023-04-10T00:28:16Z


In [42]:
t['jsonPayload.slot'].min()

6178416

In [43]:
t = t[['jsonPayload.slot', 'jsonPayload.msIntoSlot']]
t.head()

,jsonPayload.slot,jsonPayload.msIntoSlot
0,6188539,6119
1,6188539,6287
2,6188539,6459
3,6188539,5943
4,6188539,5759


In [44]:
len(t)

322

In [45]:
t = t.sort_values('jsonPayload.msIntoSlot', ascending=True).drop_duplicates('jsonPayload.slot').sort_index()
len(t)

81

In [46]:
t.head()

,jsonPayload.slot,jsonPayload.msIntoSlot
4,6188539,5759
9,6188226,4977
12,6188037,6040
15,6187750,4213
19,6187656,7610


In [47]:
n = pd.read_csv('downloaded-logs-20230409-210110.csv', delimiter=',')
nslots = list(set(n['jsonPayload.slot']))
nslots

[6182018,
 6184838,
 6184842,
 6186126,
 6187925,
 6183319,
 6185886,
 6181793,
 6182694,
 6188333,
 6186159,
 6185267,
 6179507,
 6179255,
 6187961,
 6184123,
 6185276,
 6179519,
 6180545,
 6183626,
 6183883,
 6184654,
 6188369,
 6181971,
 6186196,
 6186076,
 6183010,
 6187109,
 6181997,
 6183407,
 6181490,
 6180851,
 6180853,
 6182518,
 6182139]

In [48]:
t = t[~t['jsonPayload.slot'].isin(nslots)]
len(t)

80

In [49]:
t

,jsonPayload.slot,jsonPayload.msIntoSlot
4,6188539,5759
9,6188226,4977
12,6188037,6040
15,6187750,4213
19,6187656,7610
...,...,...
305,6179150,4662
306,6179071,4897
311,6178859,5541
314,6178687,4065


In [50]:
res = []
for s in t['jsonPayload.slot']:
    time.sleep(0.1)
    url = 'http://57.128.92.65:3500/eth/v1/beacon/headers/{}'.format(s)
    headers = {'content-type': 'application/json'}
    r = requests.get(url, headers=headers)
    try: 
        code = r.json()['code']
        print(s, False)
        res.append((s, False))
    except KeyError:
        print(s, True)
        res.append((s, True))
res = np.asarray(res)

6188539 True
6188226 False
6188037 True
6187750 True
6187656 True
6187600 False
6187462 True
6187403 True
6187379 True
6187159 True
6187043 True
6186907 True
6186794 True
6186767 True
6186720 True
6186569 True
6186567 True
6186499 True
6186425 False
6186275 True
6186077 True
6185923 True
6185887 True
6185771 True
6185633 True
6185602 False
6185311 True
6185252 True
6185205 True
6185191 True
6184869 True
6184820 True
6184719 True
6184648 True
6184367 True
6184165 True
6183964 True
6183715 True
6183459 True
6183408 True
6183135 True
6183041 True
6182868 True
6182775 True
6182499 True
6182473 True
6182431 True
6182365 False
6182304 False
6181734 False
6181440 True
6181424 True
6181387 True
6181368 True
6181194 True
6181191 True
6181167 True
6181142 False
6181105 True
6181082 True
6181022 True
6180812 True
6180757 True
6180668 True
6180578 True
6180523 True
6180272 True
6180036 True
6180010 True
6179952 True
6179778 True
6179749 True
6179662 True
6179641 True
6179444 True
6179150 True
6179

In [51]:
missed = [v[0] for v in res if v[1] == 0]

In [52]:
t[t['jsonPayload.slot'].isin(missed)]

,jsonPayload.slot,jsonPayload.msIntoSlot
9,6188226,4977
25,6187600,4038
74,6186425,4042
111,6185602,4667
196,6182365,4042
202,6182304,7006
204,6181734,5052
232,6181142,6419
321,6178416,5918


## after 6188226

In [66]:
t = pd.read_csv('downloaded-logs-20230411-153923.csv', delimiter=',').dropna(subset=['jsonPayload.slot'])
t = t.astype({'jsonPayload.slot': 'int'})
t.head()

,insertId,jsonPayload.blockHash,jsonPayload.contentLength,jsonPayload.headSlot,jsonPayload.idArg,jsonPayload.level,jsonPayload.method,jsonPayload.mevBoostV,jsonPayload.msIntoSlot,jsonPayload.msg,...,receiveTimestamp,resource.labels.cluster_name,resource.labels.container_name,resource.labels.location,resource.labels.namespace_name,resource.labels.pod_name,resource.labels.project_id,resource.type,severity,timestamp
0,arauypo8k1nt8nul,0xf29edf8fb25f069c278d264665b85dea9d7ad9a91814...,84763,6201467,NaN,warning,getPayload,-,4747,getPayload sent too late,...,2023-04-11T19:33:52.710833433Z,midori,proposer-api,europe-west1-b,default,proposer-api-7f6c678dd5-k96hx,ultra-sound-relay,k8s_container,WARNING,2023-04-11T19:33:51Z
1,zvad1q9xpl0wv7q5,0xcc2d82c661059022019101b479f018d130aeeef0adb6...,84794,6201453,NaN,warning,getPayload,v1.5.0,4193,getPayload sent too late,...,2023-04-11T19:31:08.228558067Z,midori,proposer-api,europe-west1-b,default,proposer-api-7f6c678dd5-ksxt4,ultra-sound-relay,k8s_container,WARNING,2023-04-11T19:31:03Z
2,pf0wpyk4sgv91fz1,0xcc2d82c661059022019101b479f018d130aeeef0adb6...,84794,6201453,NaN,warning,getPayload,v1.5.0,4030,getPayload sent too late,...,2023-04-11T19:31:07.738511556Z,midori,proposer-api,europe-west1-b,default,proposer-api-7f6c678dd5-k96hx,ultra-sound-relay,k8s_container,WARNING,2023-04-11T19:31:03Z
3,aevhwxej19viwp2g,0xe4a7097414bdf39e9f09a007aa02387f28a0cc02dc4f...,58487,6201342,NaN,warning,getPayload,-,4275,getPayload sent too late,...,2023-04-11T19:08:55.478489158Z,midori,proposer-api,europe-west1-b,default,proposer-api-7f6c678dd5-bt8rv,ultra-sound-relay,k8s_container,WARNING,2023-04-11T19:08:51Z
4,xatcll7rxqc3hkr3,0x61bf4ca09776985a64d603622cfd9aad8ccacc1bd80b...,82841,6201312,stakely,warning,getPayload,v1.5.0,13123,getPayload sent too late,...,2023-04-11T19:03:00.464159032Z,midori,proposer-api,europe-west1-b,default,proposer-api-7f6c678dd5-bt8rv,ultra-sound-relay,k8s_container,WARNING,2023-04-11T19:03:00Z


In [67]:
t = t[['jsonPayload.slot', 'jsonPayload.msIntoSlot']]
t.head()

,jsonPayload.slot,jsonPayload.msIntoSlot
0,6201467,4747
1,6201453,4193
2,6201453,4030
3,6201342,4275
4,6201312,13123


In [68]:
len(t)

330

In [69]:
t = t.sort_values('jsonPayload.msIntoSlot', ascending=True).drop_duplicates('jsonPayload.slot').sort_index()
len(t)

91

In [70]:
t.head()

,jsonPayload.slot,jsonPayload.msIntoSlot
0,6201467,4747
2,6201453,4030
3,6201342,4275
17,6201312,7596
19,6201251,4762


In [71]:
n = pd.read_csv('downloaded-logs-20230411-154458.csv', delimiter=',')
nslots = list(set(n['jsonPayload.slot']))
nslots

[6197250,
 6192646,
 6195211,
 6193679,
 6197268,
 6194721,
 6194217,
 6190634,
 6189613,
 6198341,
 6197829,
 6192201,
 6195278,
 6195293,
 6194785,
 6197860,
 6193256,
 6197353,
 6200943,
 6191225,
 6197376,
 6190720,
 6189186,
 6190736,
 6196369,
 6189205,
 6201499,
 6197921,
 6198434,
 6197923,
 6189749,
 6195385,
 6190275,
 6200007,
 6197965,
 6197979,
 6191323,
 6199535,
 6199030,
 6192376,
 6192891,
 6195454,
 6196478,
 6191366,
 6188815,
 6197528,
 6201120,
 6198050,
 6198051,
 6194979,
 6192422,
 6188333,
 6192438,
 6200119,
 6199619,
 6192969,
 6200139,
 6197583,
 6188369,
 6191956,
 6191986,
 6195572,
 6194570,
 6200215,
 6193050,
 6195102,
 6197157,
 6200749,
 6197678,
 6191025,
 6197684,
 6197183,
 6199748,
 6194116,
 6195659,
 6190029,
 6194128,
 6191569,
 6196690,
 6189521,
 6194153,
 6199792,
 6193653]

In [72]:
t = t[~t['jsonPayload.slot'].isin(nslots)]
len(t)

85

In [73]:
t

,jsonPayload.slot,jsonPayload.msIntoSlot
0,6201467,4747
2,6201453,4030
3,6201342,4275
17,6201312,7596
19,6201251,4762
...,...,...
326,6189538,4190
327,6189033,4115
330,6189030,4124
333,6188927,7750


In [74]:
res = []
for s in t['jsonPayload.slot']:
    time.sleep(0.1)
    url = 'http://57.128.92.65:3500/eth/v1/beacon/headers/{}'.format(s)
    headers = {'content-type': 'application/json'}
    r = requests.get(url, headers=headers)
    try: 
        code = r.json()['code']
        print(s, False)
        res.append((s, False))
    except KeyError:
        print(s, True)
        res.append((s, True))
res = np.asarray(res)

6201467 True
6201453 True
6201342 True
6201312 True
6201251 True
6201152 False
6201033 True
6200439 True
6200404 True
6200352 True
6200337 True
6200288 True
6200228 True
6200177 True
6200096 True
6200033 True
6199872 True
6199849 True
6199765 True
6199670 True
6199491 True
6199429 True
6199345 True
6199237 True
6199032 True
6198965 False
6198809 True
6198697 True
6198275 True
6198219 True
6198033 True
6197708 True
6197367 True
6197305 True
6197229 True
6196948 True
6196672 True
6196489 True
6196394 False
6196339 True
6196037 True
6195984 True
6195936 True
6195810 True
6195658 True
6195350 True
6195259 True
6195235 True
6195065 True
6194984 True
6194976 True
6194972 True
10 True
1 True
6194914 True
6194891 True
6194872 True
6194863 True
6194438 True
6194270 True
6194266 True
6194143 True
6193955 True
6193212 True
6193137 True
6193016 True
6192719 True
6192618 True
6192581 True
6192538 True
6192254 True
6192088 False
6191870 True
6191659 True
6191583 True
6191546 True
6191054 True
619070

In [75]:
missed = [v[0] for v in res if v[1] == 0]

In [76]:
t[t['jsonPayload.slot'].isin(missed)]

,jsonPayload.slot,jsonPayload.msIntoSlot
29,6201152,6347
92,6198965,4212
150,6196394,4269
272,6192088,8003
